<a href="https://colab.research.google.com/github/chaoticsoul2004/YTvideo_summarizer/blob/main/Transformers_Based_Text_Summarization_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.7 MB/s eta 0:00:00


In [2]:
# ignore warnings

import warnings
warnings.filterwarnings("ignore")

Installing Dependencies

In [3]:
! pip install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
! pip install datasets
! pip install rouge

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
ERROR: Could not find a version that satisfies the requirement torch==1.8.2+cu111 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.8.2+cu111
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user


Load dataset

In [5]:
from datasets import load_dataset
multi_news = load_dataset("multi_news", split="test")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

View Data

In [6]:
multi_news.to_pandas()

,document,summary
0,GOP Eyes Gains As Voters In 11 States Pick Gov...,– It's a race for the governor's mansion in 11...
1,\n \n \n \n UPDATE: 4/19/2001 Read Richard Met...,– It turns out Facebook is only guilty of abou...
2,It's the Golden State's latest version of the ...,– Not a big fan of Southern California? Neithe...
3,The seed for this crawl was a list of every ho...,– Why did Microsoft buy Nokia's phone business...
4,After a year in which liberals scored impressi...,– The Supreme Court is facing a docket of high...
...,...,...
5617,Tweet with a location \n \n You can add locati...,– The traditional end-of-summit group photo at...
5618,Loic Venance/AFP/Getty Images \n \n The awards...,– Sofia Coppola scored a historic victory at t...
5619,(CNN) A federal criminal investigation into a ...,– The duck boat sinking that killed 17 on a Mi...
5620,An archive of the public statements deleted by...,– Note to tweeting politicians: Watch what you...


Train test split (80:20 ratio respectively)

In [7]:
multi_news = multi_news.train_test_split(test_size=0.2)

Tokenizing Training and testing sets

In [8]:
! pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.7 MB/s eta 0:00:00


In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [10]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
tokenized_multi_news = multi_news.map(preprocess_function, batched=True)

Map:   0%|          | 0/4497 [00:00<?, ? examples/s]

Map:   0%|          | 0/1125 [00:00<?, ? examples/s]

Model Loading

In [13]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='t5-small')
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='gpt2')


model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

Defining Hyperparameters

In [ ]:
 !pip install accelerate>=0.21.1
 !pip install transformers


In [ ]:
import accelerate
accelerate.__version__

In [ ]:
!pip install accelerate>=0.2

In [14]:
training_args = Seq2SeqTrainingArguments(
output_dir="./results",
evaluation_strategy="epoch",
learning_rate=2e-5,
per_device_train_batch_size=10,
per_device_eval_batch_size=10,
weight_decay=0.01,
save_total_limit=3,
num_train_epochs=10,
fp16=True,
)


Initializing Trainer

In [15]:
trainer = Seq2SeqTrainer(
model=model,
args=training_args,
train_dataset=tokenized_multi_news["train"],
eval_dataset=tokenized_multi_news["test"],
tokenizer=tokenizer,
data_collator=data_collator,
)


In [ ]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


Train the Text Summarization Model

In [ ]:
# trainer.train()

Evaluate model on a single document

In [ ]:
document = "National Archives Yes, it's that time again, folks. It's the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you're here, why don't you sign up to follow us on Twitter. Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be growing fast enough to sustain robust job growth. The unemployment rate dipped, but mostly because more Americans stopped looking for work. The Labor Department says the economy added 120,000 jobs in March, down from more than 200,000 in each of the previous three months. The unemployment rate fell to 8.2 percent, the lowest since January 2009. The rate dropped because fewer people searched for jobs. The official unemployment tally only includes those seeking work. The economy has added 858,000 jobs since December _ the best four months of hiring in two years. But Federal Reserve Chairman Ben Bernanke has cautioned that the current hiring pace is unlikely to continue without more consumer spending."
human_summary = """– The unemployment rate dropped to 8.2% last month, but the economy only added 120,000 jobs, when 203,000 new jobs had been predicted, according to today's jobs report. Reaction on the Wall Street Journal's MarketBeat Blog was swift: "Woah!!! Bad number." The unemployment rate, however, is better news; it had been expected to hold steady at 8.3%. But the AP notes that the dip is mostly due to more Americans giving up on seeking employment."""

In [ ]:
def predict_summary(document):
  device = model.device
  tokenized = tokenizer([document], truncation =True, padding ='longest',return_tensors='pt')
  tokenized = {k: v.to(device) for k, v in tokenized.items()}
  tokenized_result = model.generate(**tokenized, max_length=300)
  tokenized_result = tokenized_result.to('cpu')
  predicted_summary = tokenizer.decode(tokenized_result[0])
  return predicted_summary

In [ ]:
document

In [ ]:
human_summary

In [ ]:
predicted_summary = predict_summary(document)
predicted_summary

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("Mayank1309/YTvideoSummarizer")

In [ ]:
# import Pytorch as torch
# trainer.save_model('/content/text_summary')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install -q youtube_transcript_api

In [ ]:
# from transformers import pipeline
# from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
youtube_video = "https://www.youtube.com/watch?v=A4OmtyaBHFE"
video_id = youtube_video.split("=")[1]
from IPython.display import YouTubeVideo
YouTubeVideo(video_id)

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install youtube_transcript_api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi


In [ ]:
YouTubeTranscriptApi.get_transcript(video_id)
transcript = YouTubeTranscriptApi.get_transcript(video_id)


In [ ]:
transcript[0:5]

In [ ]:
result = ""
for i in transcript:
    result += ' ' + i['text']
#print(result)
print(len(result))

In [ ]:
predict_summary(result)

Evaluate using Rouge Scores

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a suite of metrics for evaluating the quality of text summaries. It compares a set of reference summaries with a set of generated summaries, and computes a score based on the overlap between the two.

In [ ]:
from rouge import Rouge

def get_rouge_scores(actual_summary, predicted_summary):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, actual_summary)
    return [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']]

In [ ]:
print("key")

In [ ]:
# import validation dataset

import pandas as pd
val_data = pd.read_excel('/content/Validation_Data.xlsx')
val_data

In [ ]:
# rouge score of validation data

from tqdm import tqdm

rouge1_scores = []
rouge2_scores = []
rougel_scores = []

pred_summary_list = []

for i in tqdm(range(len(val_data))):

  doc = val_data.loc[i]['document']
  pred_summary = predict_summary(document)
  human_summary = val_data.loc[i]['summary']

  score = get_rouge_scores(human_summary, pred_summary)

  rouge1_scores.append(score[0])
  rouge2_scores.append(score[1])
  rougel_scores.append(score[2])

  pred_summary_list.append(pred_summary)

val_data["pred_summary"] = pred_summary_list

val_data['rouge1'] = rouge1_scores
val_data['rouge2'] = rouge2_scores
val_data['rougel'] = rougel_scores

val_data

In [ ]:
# average rouge 1
val_data['rouge1'].mean()

In [ ]:
# average rouge 2
val_data['rouge2'].mean()

In [ ]:
# average rouge l
val_data['rougel'].mean()

## Text Translation

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install transformers datasets evaluate sacrebleu

In [ ]:
from datasets import load_dataset
books=load_dataset("opus_books",'en-fr')
books

In [ ]:
# books.to_pandas()

In [ ]:
books['train'][334]

In [ ]:
books=books['train'].train_test_split(test_size=0.2)

In [ ]:
data

In [ ]:
books['test'][50]

In [ ]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('t5-small')

In [ ]:
def preprocess(dataset):
  input=['en'+doc['en'] for doc in dataset['translation']]
  labels=['fr'+doc['fr'] for doc in dataset['translation']]
  input_tokenized=tokenizer(input,max_length=128,truncation=True)
  labels_tokenized=tokenizer(labels,max_length=128,truncation=True)
  input_tokenized['labels']=labels_tokenized['input_ids']
  return input_tokenized


In [ ]:
tokenized_books=books.map(preprocess,batched=True)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='t5-small')

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
import accelerate
accelerate.__version__


In [ ]:
import numpy as np


def postprocess_text(preds, labels):

    preds = [pred.strip() for pred in preds]

    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):

    preds, labels = eval_preds

    if isinstance(preds, tuple):

        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)

    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

    result["gen_len"] = np.mean(prediction_lens)

    result = {k: round(v, 4) for k, v in result.items()}

    return result

In [ ]:
training_args = Seq2SeqTrainingArguments(

    output_dir="/content/my_model",

    evaluation_strategy="epoch",

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    weight_decay=0.01,

    save_total_limit=3,

    num_train_epochs=2,

    predict_with_generate=True,

    fp16=True,

    push_to_hub=True,

)

trainer = Seq2SeqTrainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_books["train"],

    eval_dataset=tokenized_books["test"],

    tokenizer=tokenizer,

    data_collator=data_collator,

    # compute_metrics=compute_metrics,

)

In [ ]:
trainer.train()